# Offline Learning


In [1]:
# Load in all your libraries and packages here!
# NOTE: you are allowed to use packages loaded below ONLY. If you want to use any other packages, please contact the TA.
import pandas as pd
import numpy as np
import itertools
import math
import random
import copy
import time
import pandas as pd
import haversine
# from haversine import haversine
import unittest
import pickle
import collections

from utils import * # this imports all helper functions in utils.py

# TODO: Replace the string "TeamName" below with your own team's name in camel case. For e.g. "AwesomeTeam"
# DO NOT RENAME THE FILE, the file name should be "{TEAM_NAME}_Policies.py"
TEAM_NAME = "poop"

In [2]:
from sklearn.linear_model import LogisticRegression
# Load data needed for offline pricing and matching
training_data = pd.read_csv("data/training_data.csv")
area_coords = pd.read_csv("data/area_lat_lon.csv").set_index('area_id')
training_data.dropna(subset=['quoted_price', 'convert_or_not'], inplace=True)
offline_data = training_data.copy()

# Build base price table
base_price_table = offline_data.groupby(['pickup_area', 'dropoff_area'])['quoted_price'].median().reset_index()
base_price_table.rename(columns={'quoted_price': 'median_price'}, inplace=True)


conversion_rates = offline_data.groupby(['pickup_area', 'dropoff_area'])['convert_or_not'].mean().reset_index()
conversion_rates.rename(columns={'convert_or_not': 'conversion_rate'}, inplace=True)
base_price_table = base_price_table.merge(conversion_rates, on=['pickup_area', 'dropoff_area'], how='left')

base_price_table['base_price'] = base_price_table['median_price'] * (1 + 0.1 * (base_price_table['conversion_rate'] - 0.5))



pair_counts = offline_data.groupby(['pickup_area', 'dropoff_area']).size().reset_index(name='count')
thresh = pair_counts['count'].quantile(0.9)
hot_pairs = pair_counts[pair_counts['count'] > thresh][['pickup_area', 'dropoff_area']]
base_price_table = base_price_table.merge(hot_pairs.assign(hot_pair=True), on=['pickup_area', 'dropoff_area'], how='left')
base_price_table['hot_pair'] = base_price_table['hot_pair'].fillna(False)
base_price_table.loc[base_price_table['hot_pair'], 'base_price'] *= 1.05

pickup_avg = offline_data.groupby('pickup_area')['quoted_price'].mean()
dropoff_avg = offline_data.groupby('dropoff_area')['quoted_price'].mean()
overall_avg = offline_data['quoted_price'].mean()

def marginal_boost(row):
    pickup_boost = pickup_avg.get(row['pickup_area'], overall_avg) - overall_avg
    dropoff_boost = dropoff_avg.get(row['dropoff_area'], overall_avg) - overall_avg
    return 0.05 * (pickup_boost + dropoff_boost)

base_price_table['base_price'] += base_price_table.apply(marginal_boost, axis=1)
base_price_table.set_index(['pickup_area', 'dropoff_area'], inplace=True)
base_price_table.to_csv("base_price_table.csv")